In [59]:
b=np.load('./faults/faulttrain1.npz')
b['arr_0'].shape

(100, 3174, 1537)

In [60]:
c = np.load("./faults/seistrain1.npz")
c['arr_0'].shape

(100, 3174, 1537)

In [38]:
c['arr_0'][50:80].shape

(30, 3174, 1537)

In [39]:
trainseis = c['arr_0'][:50]
valseis = c['arr_0'][50:80]
testseis = c['arr_0'][80:]

In [40]:
np.save('./faults/trainseis.npy',trainseis)
np.save('./faults/valseis.npy',valseis)
np.save('./faults/testseis.npy',testseis)

In [44]:
trainfault = b['arr_0'][:50]
valfault = b['arr_0'][50:80]
testfault = b['arr_0'][80:]

In [45]:
np.save('./faults/ml_data/trainfault.npy',trainfault)
np.save('./faults/ml_data/valfault.npy',valfault)
np.save('./faults/ml_data/testfault.npy',testfault)

In [1]:
# coding: utf-8

import time
import cv2 
import math
from itertools import compress,product
# from skimage import color # rgb order
from skimage.util.shape import view_as_windows
from typing import Tuple

import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
plt.switch_backend('agg')
import numpy as np  
import scipy.signal
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
from torch.autograd import Variable

import matplotlib.pyplot as plt
import random
import os
from os.path import isfile, join

from albumentations import (
    HorizontalFlip, VerticalFlip, CLAHE,
    ShiftScaleRotate, OpticalDistortion, GridDistortion, ElasticTransform,
    RandomBrightnessContrast, IAASharpen, IAAEmboss, Flip, OneOf, Compose)

import warnings
warnings.filterwarnings("ignore")



# In[2]:
def cross_entropy_loss_HED(prediction, label):
    label = label.long()
    mask = label.float()
    num_positive = torch.sum((mask == 1).float()).float()
    num_negative = torch.sum((mask == 0).float()).float()

    mask[mask == 1] = 1.0 * num_negative / (num_positive + num_negative)
    mask[mask == 0] = 1.0 * num_positive / (num_positive + num_negative)
    cost = torch.nn.functional.binary_cross_entropy(
            prediction.float(),label.float(), weight=mask, reduce=False)
    return torch.sum(cost) #torch.mean(cost)

def cross_entropy_loss_RCF(prediction, label):
    label = label.long()
    mask = label.float()
    num_positive = torch.sum((mask==1).float()).float()
    num_negative = torch.sum((mask==0).float()).float()

    mask[mask == 1] = 1.0 * num_negative / (num_positive + num_negative)
    mask[mask == 0] = 1.1 * num_positive / (num_positive + num_negative)
    cost = torch.nn.functional.binary_cross_entropy(
            prediction.float(),label.float(), weight=mask, reduce=False)
    return torch.sum(cost) #torch.mean(cost)


def split_Image(bigImage,isMask,top_pad,bottom_pad,left_pad,right_pad,splitsize,stepsize,vertical_splits_number,horizontal_splits_number):
#     print(bigImage.shape)
    if isMask==True:
        arr = np.pad(bigImage,((top_pad,bottom_pad),(left_pad,right_pad)),"reflect")
        splits = view_as_windows(arr, (splitsize,splitsize),step=stepsize)
        splits = splits.reshape((vertical_splits_number*horizontal_splits_number,splitsize,splitsize))
    else: 
        arr = np.pad(bigImage,((top_pad,bottom_pad),(left_pad,right_pad),(0,0)),"reflect")
        splits = view_as_windows(arr, (splitsize,splitsize,3),step=stepsize)
        splits = splits.reshape((vertical_splits_number*horizontal_splits_number,splitsize,splitsize,3))
    return splits # return list of arrays.


def recover_Image(patches: np.ndarray, imsize: Tuple[int, int, int], left_pad,right_pad,top_pad,bottom_pad, overlapsize):
#     patches = np.squeeze(patches)
    assert len(patches.shape) == 5

    i_h, i_w, i_chan = imsize
    image = np.zeros((i_h+top_pad+bottom_pad, i_w+left_pad+right_pad, i_chan), dtype=patches.dtype)
    divisor = np.zeros((i_h+top_pad+bottom_pad, i_w+left_pad+right_pad, i_chan), dtype=patches.dtype)

#     print("i_h, i_w, i_chan",i_h, i_w, i_chan)
    n_h, n_w, p_h, p_w,_= patches.shape
    
    o_w = overlapsize
    o_h = overlapsize

    s_w = p_w - o_w
    s_h = p_h - o_h

    for i, j in product(range(n_h), range(n_w)):
        patch = patches[i,j]
        image[(i * s_h):(i * s_h) + p_h, (j * s_w):(j * s_w) + p_w] += patch
        divisor[(i * s_h):(i * s_h) + p_h, (j * s_w):(j * s_w) + p_w] += 1

    recover = image / divisor
    return recover[top_pad:top_pad+i_h, left_pad:left_pad+i_w]

def recover_Image2(patches: np.ndarray, imsize: Tuple[int, int, int], left_pad,right_pad,top_pad,bottom_pad, overlapsize):
#     patches = np.squeeze(patches)
    assert len(patches.shape) == 5

    i_h, i_w, i_chan = imsize
    image = np.zeros((i_h+top_pad+bottom_pad, i_w+left_pad+right_pad, i_chan), dtype=patches.dtype)
    divisor = np.zeros((i_h+top_pad+bottom_pad, i_w+left_pad+right_pad, i_chan), dtype=patches.dtype)

#     print("i_h, i_w, i_chan",i_h, i_w, i_chan)
    n_h, n_w, p_h, p_w,_= patches.shape
    
    o_w = overlapsize
    o_h = overlapsize

    s_w = p_w - o_w
    s_h = p_h - o_h

    for i, j in product(range(n_h), range(n_w)):
        patch = patches[i,j]
        image[(i * s_h):(i * s_h) + p_h, (j * s_w):(j * s_w) + p_w] += patch
#         divisor[(i * s_h):(i * s_h) + p_h, (j * s_w):(j * s_w) + p_w] += 1

    recover = image / 4
    return recover[top_pad:top_pad+i_h, left_pad:left_pad+i_w]

#https://github.com/Vooban/Smoothly-Blend-Image-Patches
def spline_window(window_size, power=2):
    """
    Squared spline (power=2) window function:
    https://www.wolframalpha.com/input/?i=y%3Dx**2,+y%3D-(x-2)**2+%2B2,+y%3D(x-4)**2,+from+y+%3D+0+to+2
    """
    intersection = int(window_size/4)
    wind_outer = (abs(2*(scipy.signal.triang(window_size))) ** power)/2
    wind_outer[intersection:-intersection] = 0

    wind_inner = 1 - (abs(2*(scipy.signal.triang(window_size) - 1)) ** power)/2
    wind_inner[:intersection] = 0
    wind_inner[-intersection:] = 0

    wind = wind_inner + wind_outer
    wind = wind / np.average(wind)
    return wind


cached_2d_windows = dict()
def window_2D(window_size, power=2):
    """
    Make a 1D window function, then infer and return a 2D window function.
    Done with an augmentation, and self multiplication with its transpose.
    Could be generalized to more dimensions.
    """
    # Memoization
    global cached_2d_windows
    key = "{}_{}".format(window_size, power)
    if key in cached_2d_windows:
        wind = cached_2d_windows[key]
    else:
        wind = spline_window(window_size, power)
        wind = np.expand_dims(np.expand_dims(wind, -1), -1)
        wind = wind * wind.transpose(1, 0, 2)
        cached_2d_windows[key] = wind
    return wind

def crop(variable, th, tw):
        h, w = variable.shape[2], variable.shape[3]
        x1 = int(round((w - tw) / 2.))
        y1 = int(round((h - th) / 2.))
        return variable[:, :, y1 : y1 + th, x1 : x1 + tw]
    
def strong_aug(p=1):
    return OneOf([
        ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=1),
        IAASharpen(p=1),
        IAAEmboss(p=1),
        RandomBrightnessContrast(p=1),
        HorizontalFlip(p=1),
        VerticalFlip(p=1),
        Compose([VerticalFlip(p=1), HorizontalFlip(p=1)]),
        ElasticTransform(p=1, alpha=400, sigma=400 * 0.05, alpha_affine=400 * 0.03),
        GridDistortion(p=1),
        OpticalDistortion(p=1)
    ], p=p)

#idea from https://www.kaggle.com/iezepov/fast-iou-scoring-metric-in-pytorch-and-numpy
SMOOTH = 1e-6
def iou_pytorch(outputs: torch.Tensor, labels: torch.Tensor):
    # You can comment out this line if you are passing tensors of equal shape
    # But if you are passing output from UNet or something it will most probably
    # be with the BATCH x 1 x H x W shape
    
    outputs = outputs.squeeze(1)  # BATCH x 1 x H x W => BATCH x H x W

    intersection = (outputs & labels).float().sum((1, 2))  # Will be zero if Truth=0 or Prediction=0
    union = (outputs | labels).float().sum((1, 2))  # Will be zzero if both are 0

    iou = (intersection + SMOOTH) / (union + SMOOTH)  # We smooth our devision to avoid 0/0
    return iou

def iou_numpy(outputs: np.array, labels: np.array):

    intersection = (outputs & labels).sum((0,1))
    union = (outputs | labels).sum((0,1))
    iou = (intersection + SMOOTH) / (union + SMOOTH)
    return iou  # Or thresholded.mean()


def crop2(variable, th, tw): # this is for crop center when outputs are 96*96
        h, w = variable.shape[-2], variable.shape[-1]
        x1 = int(round((w - tw) / 2.))
        y1 = int(round((h - th) / 2.))
        return variable[:, :, y1 : y1 + th, x1 : x1 + tw]

#https://blog.csdn.net/qq_15602569/article/details/79565402 
def acc_metrics(outputs, labels):
    TP=0
    TN=0
    FP=0
    FN=0
    # TP    predict 和 label 同时为1
    TP += ((outputs == 1) & (labels == 1)).sum()
    # TN    predict 和 label 同时为0
    TN += ((outputs == 0) & (labels == 0)).sum()
    # FN    predict 0 label 1
    FN += ((outputs == 0) & (labels == 1)).sum()
    # FP    predict 1 label 0
    FP += ((outputs == 1) & (labels == 0)).sum()
    p = TP / (TP + FP)
    r = TP / (TP + FN)
    F1 = 2 * r * p / (r + p)
    acc = (TP + TN) / (TP + TN + FP + FN)
    
    return p,r,F1,acc

/var/folders/kl/gztfr05x5r9gbmdhs041ny0h0000gn/T/ipykernel_57549/2091789324.py:12: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-white')


In [69]:
from functions import *
import torch
import numpy as np
import random
import cmapy
import time
def process_raw_data(data_type, npy_data_name, npy_label_name, split_size, step_size,length):
    
    

    # setting random seed
    torch.manual_seed(1)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(1)
    random.seed(1)
    time1 = time.time()

    print("started reading")
    data_folder = ""
    processed_data_folder = "{}./processed_data".format(data_folder)
    original_data_folder = "{}./faults/ml_data".format(data_folder)


    # loading training set
    seismic_path = '{}/{}.npy'.format(original_data_folder,npy_data_name)
    label_path = '{}/{}.npy'.format(original_data_folder,npy_label_name)
    t_start = time.time()
    seismic = np.load(seismic_path)
    fault = np.load(label_path)
    print("loaded data in {} sec".format(time.time()-t_start))

    # converting to IL, Z, XL order
    seismic = np.moveaxis(seismic,-2,-1) 
    fault = np.moveaxis(fault,-2,-1)
    seismic = (seismic-seismic.min(axis=(1,2), keepdims=True))/(seismic.max(axis=(1,2), keepdims=True)-seismic.min(axis=(1,2), keepdims=True))

    print(seismic.shape, fault.shape)
    print(seismic.max(),seismic.min(), fault.max(), fault.min())


    IL, Z, XL = fault.shape

    im_height = Z
    im_width = XL
    splitsize = split_size
    stepsize = step_size
    overlapsize = splitsize-stepsize
    pixel_max = int(0.03*splitsize*splitsize)


    hori_splits_number = int(np.ceil((im_width-overlapsize)/stepsize))
    print("hori_splits", hori_splits_number)
    width_after_pad = stepsize*hori_splits_number+overlapsize
    print("width_after_pad", width_after_pad)
    left_pad = int((width_after_pad-im_width)/2)
    right_pad = width_after_pad-im_width-left_pad
    print("l_pad,r_pad",left_pad,right_pad)

    vertical_splits_number = int(np.ceil((im_height-overlapsize)/stepsize))
    print("vertical_splits",vertical_splits_number)
    height_after_pad = stepsize*vertical_splits_number+overlapsize
    print("height_after_pad",height_after_pad)
    top_pad = int((height_after_pad-im_height)/2)
    bottom_pad = height_after_pad-im_height-top_pad
    print("top_pad,bot_pad", top_pad,bottom_pad)


    t_start = time.time()
    X = []
    Y = []
    for i in range(0,length,1):
        mask = fault[i]
        splits = split_Image(mask, True,top_pad,bottom_pad,left_pad,right_pad,splitsize,stepsize,vertical_splits_number,hori_splits_number)
        t = (splits.sum((1,2)) < pixel_max)
        no_label_element_index = list(compress(range(len(t)), t))
        splits = np.delete(splits, no_label_element_index,0) 
        Y.extend(splits)

        img = seismic[i]
        splits = split_Image(img, True,top_pad,bottom_pad,left_pad,right_pad,splitsize,stepsize,vertical_splits_number,hori_splits_number)
        splits = np.delete(splits, no_label_element_index,0)
        X.extend(splits)

    print(len(Y))
    print(len(X))
    print(X[0].shape)
    print("read images in {} sec".format(time.time()-t_start))
    
    #creating directory if not already made, else leave it as it is
    directory = "{}/{}/seismic".format(processed_data_folder,data_type)
    if not os.path.exists(directory):
        os.makedirs(directory)
    directory = "{}/{}/annotation".format(processed_data_folder,data_type)
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    #saving all_images as a single npy
    np.save("{}/{}/seismic.npy".format(processed_data_folder, data_type),X)
    np.save("{}/{}/annotation.npy".format(processed_data_folder, data_type),Y)
    
    #saving_individual_images as npy
    for i in range(len(X)):
        np.save("{}/{}/seismic/{}.npy".format(processed_data_folder, data_type,i),X[i])
        np.save("{}/{}/annotation/{}.npy".format(processed_data_folder, data_type,i),Y[i])

In [62]:
#processing_training_data
process_raw_data('train', 'trainseis', 'trainfault', 128, 96,50)

start reading......
load in 10.301100730895996 sec
(50, 1537, 3174) (50, 1537, 3174)
1.0 0.0 True False
491
horizontal_splits_number 33
width_after_pad 3200
left_pad,right_pad 13 13
vertical_splits_number 16
height_after_pad 1568
top_pad,bottom_pad 15 16
2894
2894
(128, 128)
read images in 1.0883738994598389 sec


In [63]:
#processing_validation_data
process_raw_data('val', 'valseis', 'valfault', 512, 256,30)

start reading......
load in 2.0827808380126953 sec
(30, 1537, 3174) (30, 1537, 3174)
1.0 0.0 True False
7864
horizontal_splits_number 12
width_after_pad 3328
left_pad,right_pad 77 77
vertical_splits_number 6
height_after_pad 1792
top_pad,bottom_pad 127 128
360
360
(512, 512)
read images in 1.5873818397521973 sec


In [68]:
#processing_test_data
process_raw_data('test', 'testseis', 'testfault', 768, 256,20)

started reading
loaded data in 0.1453409194946289 sec
(20, 1537, 3174) (20, 1537, 3174)
1.0 0.0 True False
hori_splits_number 11
width_after_pad 3328
left_pad,right_pad 77 77
vertical_splits_number 5
height_after_pad 1792
top_pad,bottom_pad 127 128
295
295
(768, 768)
read images in 2.6631052494049072 sec


In [56]:
# saves the image for viewing training, validation and test set
def show_image(i,data_type):
    seis = np.load("{}/{}/seismic/{}.npy".format('./processed_data', data_type, i))
    mask = np.load("{}/{}/annotation/{}.npy".format('./processed_data', data_type, i))
    plt.figure(figsize=(6,4))
    plt.imshow(seis,'gray')
    plt.imshow(mask, 'gray', alpha=0.4)
    plt.savefig("{}myfilename.png".format(data_type), dpi=200)
    plt.axis('off')


In [58]:
show_image(12,'train')
show_image(12,'val')
show_image(12,'test')